In [16]:
import pandas as pd

data = pd.read_csv('../data/raw_analyst_ratings.csv')


data['headline_length'] = data['headline'].apply(len)
print(data['headline_length'].describe())

publisher_count = data['publisher'].value_counts()
print(publisher_count)


count    1.407328e+06
mean     7.312051e+01
std      4.073531e+01
min      3.000000e+00
25%      4.700000e+01
50%      6.400000e+01
75%      8.700000e+01
max      5.120000e+02
Name: headline_length, dtype: float64
publisher
Paul Quintaro        228373
Lisa Levin           186979
Benzinga Newsdesk    150484
Charles Gross         96732
Monica Gerson         82380
                      ...  
Matthew Ely               1
Frank Ochoa               1
Jeremie Capron            1
Marvin Dumont             1
Igor Gonta                1
Name: count, Length: 1034, dtype: int64
